In [35]:
from ase import Atoms
import numpy as np

In [36]:
np.random.seed(1001)

questions for Niels: 1. size of the sample; 2. normal direction; 3. lattice constant

In [37]:
latticeC = 3.463

In [38]:
size_x = 500 # unit: Angstrom
size_y = 500
size_z = 100

In [39]:
# generate a simple slab along (0,0,1)
atoms = Atoms('Ta2', positions=[(0,0,0),(latticeC/2.,latticeC/2, latticeC/2.)],
             cell=(latticeC, latticeC, latticeC))

In [40]:
repeat_x = int(size_x / latticeC)
repeat_y = int(size_y / latticeC)
repeat_z = int(size_z / latticeC)

In [41]:
atoms = atoms.repeat((repeat_x, repeat_y, repeat_z))

In [42]:
indices = np.arange(len(atoms))

In [43]:
np.random.shuffle(indices, )

In [44]:
comp_Ta = 0.093 
comp_Ti = 0.332
comp_Zr = 0.33
comp_Hf = 0.245

In [45]:
indices_Ta = indices[:int(comp_Ta*len(atoms))]
indices_Ti = indices[int(comp_Ta*len(atoms)):int((comp_Ta+comp_Ti)*len(atoms))]
indices_Zr = indices[int((comp_Ta+comp_Ti)*len(atoms)):int((comp_Ta+comp_Ti+comp_Zr)*len(atoms))]
indices_Hf = indices[int((comp_Ta+comp_Ti+comp_Zr)*len(atoms)):]

In [46]:
symbols = atoms.get_chemical_symbols()

In [47]:
symbols = np.array(symbols)

In [48]:
symbols[indices_Ta] = 'Ta'
symbols[indices_Ti] = 'Ti'
symbols[indices_Zr] = 'Zr'
symbols[indices_Hf] = 'Hf'

In [49]:
atoms.set_chemical_symbols(list(symbols))

In [50]:
atoms

Atoms(symbols='Hf284498Ta107993Ti385523Zr383202', pbc=False, cell=[498.672, 498.672, 96.964])

Questions for Niels or Christian: how to choose thermal_sigma for Debye–Waller effect?

In [51]:
def ASE2PrismaticXYZ(atoms, filename='prismatic.XYZ',
                     headline='Prismatic input file',
                     occupancy_number=1.,
                     thermal_sigma=0.1):
    """
    write ase Atoms object to XYZ file for Prismatic.
    this function only supports orthogonal supercell.
    
    Input
    -----------------
    atoms..............ase Atoms object
    occupancy_number...float or list, value between 0 and 1 that 
                       specifies the likelihood that an atom exists 
                       at that site.
                       if float, assign this value to each atom. 
                       if list, the i-th entry is the occupancy number of the i-th atom.
    thermal_sigma......float or list, parameters for Debye-Waller effect.
                       Unit: Angstrom.
                       if float, assign this value to each atom. 
                       if list, the i-th entry is the standard deviation
                       of random thermal motion for the i-th atom.
    """
    
    outf = open(filename, 'w',  encoding='utf8')
    outf.write(headline)
    outf.write('\n')
    cell = atoms.get_cell()
    outf.write('      %12.6f %12.6f %12.6f\n' % (cell[0,0], cell[1,1], cell[2,2]))
    positions = atoms.get_positions()
    atomic_numbers = atoms.get_atomic_numbers()
    if not hasattr(occupancy_number, "__len__"):
        occupancy_number = occupancy_number * np.ones(len(atoms))
    if not hasattr(thermal_sigma, "__len__"):
        thermal_sigma = thermal_sigma * np.ones(len(atoms))
    for i in range(len(atoms)):
        outf.write('%d %- 12.6f %- 12.6f %- 12.6f %7.6f %- 12.6f\n' % (atomic_numbers[i],
                 positions[i,0],positions[i,1],positions[i,2],occupancy_number[i],thermal_sigma[i]))
    outf.write('-1')
    outf.close()

In [52]:
ASE2PrismaticXYZ(atoms)